In [1]:
import time
import os
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import scipy as sc
import numpy as np
import requests

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
import chromedriver_binary  # Adds chromedriver binary to path

In [ ]:
! pip install selenium

In [ ]:
pip install chromedriver-binary

In [ ]:
pip install webdriver-manager

In [2]:
def load_soup_object(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.content,"html.parser")
    #soup = BeautifulSoup(open(html_file_name),"html.parser")
    return soup

In [3]:
#def initialize_arraies():
aniNames=[]
formats=[]
number_of_episodes=[]
episode_duration=[]
status=[]
studios=[]
favorites=[]
popularity=[]
genres=[]
mean_score=[]
source=[]
creator=[]

completed=[]
current=[]
planning=[]
paused=[]
dropped=[]

arrayOfArrays = ([aniNames, formats, number_of_episodes, episode_duration,
                  status, studios, favorites,popularity,genres,mean_score,
                  source,creator,completed,current,planning,paused,dropped])

In [4]:
def scrap(url):
    #ani_soup = load_soup_object('https://anilist.co/search/anime/popular')
    ani_soup = BeautifulSoup(url)

    count = 0
    mtag = ani_soup.find_all("div",attrs={"class":"media-card"})
    for t in mtag:
        count+=1
        anime_name = t.find('a',attrs={"class":"title"}).get_text().replace('\n','').replace('\t','')
        aniNames.append(anime_name)

        ani_url = 'https://anilist.co' + t.find('a',attrs={"class":"title"}).get('href')
        soup = load_soup_object(ani_url)
        anime_tag = soup.find_all("div",attrs={"class":"data-set"})
        for x in anime_tag:
            try:
                div_type = x.find("div",attrs={"class":"type"}).get_text()
                div_value = x.find("div",attrs={"class":"value"}).get_text()
            except:
                continue

            if(div_type == 'Format'):
                formats.append(div_value.replace('\n',''))

            if(div_type == 'Episodes'):
                number_of_episodes.append(div_value)

            if('Duration' in div_type):
                episode_duration.append(div_value)
                
            if(div_type == 'Status'):
                status.append(div_value)    
                
            if(div_type == 'Mean Score'):
                mean_score.append(div_value.replace('%',''))

            if(div_type == 'Popularity'):
                popularity.append(div_value)
                
            if(div_type == 'Favorites'):
                favorites.append(div_value)

            if(div_type == 'Studios'):
                studios.append(x.find("a").get_text().replace("\n",""))

            if(div_type == 'Source'):
                source.append(div_value)

            if(div_type == 'Genres'):
                generes_string = ''
                for g in x.find_all("span"):
                    generes_string += g.get_text() + '|'
                generes_string = generes_string[:-1]
                genres.append(generes_string)


        status_tag = soup.find_all("div",attrs={"class":"status"})
        for l in status_tag:
            status_name = l.find("div",attrs={"class":"name"}).get_text()
            status_value = l.find("div",attrs={"class":"amount"}).get_text().replace("Users","").replace("\n","")

            if('Completed' in status_name):
                completed.append(status_value)

            if('Current' in status_name):
                current.append(status_value)

            if('Planning' in status_name):
                planning.append(status_value)

            if('Paused' in status_name):
                paused.append(status_value)

            if('Dropped' in status_name):
                dropped.append(status_value)
                
        roles = soup.find_all('div',attrs={"class":"view-staff"}) #role-card view-staff small
        try:
            creator.append(roles[0].find('div',attrs={"class":"name"}).get_text().replace('\n',''))
        except:
            pass
         
        #after every anime check if there is any missing value and if it is missing, insert NaN
        for i in range(17):
            if((len(arrayOfArrays[i])<count)):
                arrayOfArrays[i].append(np.nan)
                
        time.sleep(1)

In [6]:
#https://stackoverflow.com/questions/53701759/scroll-with-keys-page-down-in-selenium-python
driver=webdriver.Chrome(ChromeDriverManager().install())
driver.get("https://anilist.co/search/anime/popular")
timeout = time.time() + 60*20  
while True:
    driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
    time.sleep(3)
    if time.time() > timeout:
        #initialize_arrays()
        scrap(driver.page_source)
        break

#print(len(aniNames), len(formats), len(number_of_episodes), len(episode_duration),len(status),... len(popularity),len(favorites),len(studios), len(genres), len(mean_score), len(source),len(creator),len(completed),len(current),len(planning),len(paused),len(dropped))

data = {'anime_name': aniNames, 'Format':formats, 'Number Of Episodes':number_of_episodes,
        "Episode Duration":episode_duration, "status":status, "Popularity":popularity,
        "favorites":favorites,"Studios":studios, "Genres":genres, "mean_score":mean_score,
        "source":source,"Creator":creator,'Completed':completed,'Current':current,
        'Planning':planning,'Paused':paused,'Dropped':dropped}

df = pd.DataFrame(data)
driver.quit()  # Close the browser



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [C:\Users\97254\.wdm\drivers\chromedriver\win32\96.0.4664.45\chromedriver.exe] found in cache
<ipython-input-6-878a49089943>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(ChromeDriverManager().install())
<ipython-input-6-878a49089943>:6: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)


40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40


In [7]:
df

,anime_name,Format,Number Of Episodes,Episode Duration,status,Popularity,favorites,Studios,Genres,mean_score,source,Creator,Completed,Current,Planning,Paused,Dropped
0,Shingeki no Kyojin,TV,25,24 mins,Finished,523588,47765,Wit Studio,Action|Drama|Fantasy|Mystery,85%,Manga,Hajime Isayama,438199,31047,36867,9198,8277
1,DEATH NOTE,TV,37,23 mins,Finished,476020,35757,MADHOUSE,Mystery|Psychological|Supernatural|Thriller,84%,Manga,Tsugumi Ooba,370348,26999,47094,16831,14748
2,Boku no Hero Academia,TV,13,24 mins,Finished,472686,23800,bones,Action|Adventure|Comedy,79%,Manga,Kouhei Horikoshi,389491,29221,39180,7503,7291
3,Kimetsu no Yaiba,TV,26,24 mins,Finished,470002,40520,ufotable,Action|Adventure|Drama|Fantasy|Supernatural,85%,Manga,Koyoharu Gotouge,363558,41238,48696,9883,6627
4,HUNTER×HUNTER (2011),TV,148,24 mins,Finished,434820,56940,MADHOUSE,Action|Adventure|Fantasy,90%,Manga,Yoshihiro Togashi,259415,65431,72202,28874,8898
5,One Punch Man,TV,12,24 mins,Finished,397841,19414,MADHOUSE,Action|Comedy|Sci-Fi|Supernatural,83%,Manga,ONE,333152,15200,35839,7669,5981
6,Jujutsu Kaisen,TV,24,24 mins,Finished,390144,40243,MAPPA,Action|Drama|Supernatural,87%,Manga,Gege Akutami,235705,81512,58307,10487,4133
7,Tokyo Ghoul,TV,12,24 mins,Finished,388224,15229,Studio Pierrot,Action|Drama|Horror|Mystery|Psychological|Supe...,76%,Manga,Sui Ishida,301296,15088,54071,7707,10062
8,Sword Art Online,TV,25,23 mins,Finished,387753,14277,A-1 Pictures,Action|Adventure|Fantasy|Romance,69%,Light Novel,Reki Kawahara,316123,12668,34335,6303,18324
9,Hagane no Renkinjutsushi: FULLMETAL ALCHEMIST,TV,64,25 mins,Finished,375601,40619,bones,Action|Adventure|Drama|Fantasy,90%,Manga,Hiromu Arakawa,249154,23765,80630,15906,6146
